In [3]:
import os
import numpy as np
import csv
import pandas as pd
os.chdir('/Users/ritwikdeshpande/DLH/Project/CS598_DLH_Project/')
from dataset.preprocessing.word2vec_embeddings_gen import Word2VecFeatureGeneration
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold

In [2]:
class DecisionTree:
    def __init__(self, x_train, y_train, x_test, y_test):
        self.dtc = DecisionTreeClassifier(splitter='random', random_state=42)
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test
        # print(self.x_train.shape, self.y_train.shape, self.x_test.shape, self.y_test.shape)
        
    def train(self):
        self.dtc.fit(self.x_train, self.y_train)

    def test_and_evaluate(self):
        y_pred = self.dtc.predict(self.x_test)
        f1_macro = f1_score(self.y_test, y_pred, average='macro')
        f1_micro = f1_score(self.y_test, y_pred, average='micro')
        #print(f"Macro F1 score: {f1_macro} and Micro F1 Score {f1_micro}")
        return f1_macro, f1_micro

In [4]:
morbidities = ['Asthma', 'CAD', 'CHF', 'Depression', 'Diabetes', 'Gallstones', 'GERD', 'Gout', 'Hypercholesterolemia', 'Hypertension', 'Hypertriglyceridemia', 'OA', 'Obesity', 'OSA', 'PVD', 'Venous-Insufficiency']

column_headings = ["Morbidity Class", "DT_Macro F1", "DT_Micro F1"]

with open("./results/word-embeddings-features/performance_DT.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow([column_headings[0], column_headings[1], column_headings[2]])
    

In [7]:
all_f1_macro_scores = []
all_f1_micro_scores = []

for morbidity in morbidities:
    print(morbidity)
    train_preprocessed_df = pd.read_csv('./dataset/train/train_data_intuitive_preprocessed.csv')
    train_preprocessed_df = train_preprocessed_df[train_preprocessed_df[morbidity].isin([1.0, 0.0])]

    X, Y, words = Word2VecFeatureGeneration(train_preprocessed_df, morbidity).word2vec_matrix_gen()
    X = np.average(X, axis=1)
    # add KFold cross validation
    skf = KFold(n_splits=10, shuffle=True, random_state=42)

    f1_macro_list = []
    f1_micro_list = []
    for train_idx, val_idx in skf.split(X, Y):
        X_train_fold, Y_train_fold = X[train_idx], Y[train_idx]
        X_val_fold, Y_val_fold = X[val_idx], Y[val_idx]

        # Training RF using TF-IDF Representation
        dt_obj = DecisionTree(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold)
        dt_obj.train()

        f1_macro, f1_micro = dt_obj.test_and_evaluate()

        f1_macro_list.append(f1_macro)
        f1_micro_list.append(f1_micro)

    f1_macro = np.mean(f1_macro_list)
    f1_micro = np.mean(f1_micro_list)
    print(f"Macro F1 score: {f1_macro} and Micro F1 Score {f1_micro}")

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro, f1_micro]
    all_f1_macro_scores.append(f1_macro)
    all_f1_micro_scores.append(f1_micro)

    with open("./results/word-embeddings-features/performance_DT.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)


with open("./results/word-embeddings-features/performance_DT.csv", "a", newline="") as file:
    writer = csv.writer(file)
    row = ["Overall-Average"]
    row.extend([sum(all_f1_macro_scores)/len(all_f1_macro_scores),  sum(all_f1_micro_scores)/len(all_f1_micro_scores) ])
    writer.writerow(row)

Asthma
(572, 3570, 100) (572,) Counter({0.0: 502, 1.0: 70})
Macro F1 score: 0.5434235561115689 and Micro F1 Score 0.798790078644888
CAD
(548, 3570, 100) (548,) Counter({1.0: 325, 0.0: 223})
Macro F1 score: 0.6063191177433815 and Micro F1 Score 0.6241077441077441
CHF
(243, 3122, 100) (243,) Counter({1.0: 243})
Macro F1 score: 1.0 and Micro F1 Score 1.0
Depression
(582, 3570, 100) (582,) Counter({0.0: 460, 1.0: 122})
Macro F1 score: 0.539344780264747 and Micro F1 Score 0.6805084745762712
Diabetes
(567, 3570, 100) (567,) Counter({1.0: 396, 0.0: 171})
Macro F1 score: 0.5564809551025718 and Micro F1 Score 0.6349310776942356
Gallstones
(593, 3570, 100) (593,) Counter({0.0: 506, 1.0: 87})
Macro F1 score: 0.4608706165537252 and Micro F1 Score 0.7134180790960453
GERD
(487, 3570, 100) (487,) Counter({0.0: 372, 1.0: 115})
Macro F1 score: 0.49071259963151076 and Micro F1 Score 0.6222789115646259
Gout
(596, 3570, 100) (596,) Counter({0.0: 518, 1.0: 78})
Macro F1 score: 0.5442442629482552 and Micro 